# DataSet Preparation Utils
by Wael Radwan

#### Library used to prepare datasets from data
continans various tools to prepare the training dataset and testing dataset

mainly used to randomize files of the dataset
divide the dataset into traininig and testing dataset
randomize the test files

<b> the main idea here is to load the dataset into classified directories then use this utility to create train/test dataset with the randomization </b>

In [1]:
import os 
import re
import random
import ntpath
import shutil
import pandas as pd
import numpy as np

#### Input description
input to this tool is a dataset of files divided into classes in the following order
- class_1:
    - file_1
    - file_2
    - .
    - .
- class_2
    - file_1
    - file_2
    - .
    - .
- .
- .


- class_n 
    - file_1
    - file_2
    - .
    - .


#### Output description
output shoud be the data divided into train and test sets  in the following order

- train
    - class_1_001
    - class_1_002
    - .
    - .
    - class_2_001
    - class_2_002
    - .
    - .
    - .
    - .
    - class_n_001
    - class_n_002
    - .
    - .
 
- test
    - class_1_001
    - class_1_002
    - .
    - .
    - class_2_001
    - class_2_002
    - .
    - .
    - .
    - .
    - class_n_001
    - class_n_002
    - .
    - .
 
    - class_label.csv (a csv file contains the filename and the label so it can be used for testing comparison)

#### create panda from the files dataset
mainly used for the test dataset to generate the file name and the label based on the label criteria
randomize is set to true if the files are ranomized in the dataset instead of ordered

In [2]:
def list_files_with_labels(path, randomize=False):
    dataset = []
    files = sorted(os.listdir(path))
    if not files:
        return dataset
    file = [os.path.splitext(x)[0] for x in files]
    ext = [os.path.splitext(x)[-1] for x in files]
    class_name = ntpath.basename(path)
    

    # create panda dataframe for file sorted by their name

    dataset = pd.DataFrame(data={"file":file,"ext":ext, "label":class_name})
    dataset = dataset[['file', 'ext', 'label']]

    # if randomize is set to True then shuffle the dataset dataframe
    if (randomize) :
        dataset= dataset.sample(frac=1)
        dataset = dataset.sample(frac=1).reset_index(drop=True)

    return dataset


#### create list of the dataset with new name for each file based on a sequence
randomize or order the files in the dataset then create a new name based on sequence number

- path      : path to the class
- seqlen    : is the lenght of the sequence so if seqlen = 3 the new file names follow the sequence 001, 002, 003, ...
- randomize : either to randomize the files in the class set or not (default is not to randomize) 
- retrun panda structure with new file name 

In [3]:
def list_with_new_file_name_seq(path,  seqlen, randomize=False,):
    df = []  
    dataset = list_files_with_labels( path , randomize)
    #if not dataset:
     #   return
    
    # recreate index in case the dataset has been randomize
    # so the seq_number will be reflected in out file_name
    df = dataset.sample(frac=1).reset_index(drop=True)
    
    # new file name will be classid_seq(of seq_len).ext
    # for example 
    # 1_0001.mp3
    # 1_0002.mp3
    df['newfilename'] = df['label']+'_'+(df.index + 1).astype(str).str.zfill(seqlen) + df.ext
    
    # df.index = df.index.map(str) 
    return df

#### Rename files 
After creating the dataframe with proposed renamed files then do the rename files in the dataset
then rename all the files to _tmp files in order not to overwrite files that already exist 


In [4]:
def rename_files_dataset(path, seqlen):
    # first list the files in the input dir and put the porposed name in the dataframe
    data = list_with_new_file_name_seq( path,seqlen)
    #if not data:
     #   return
    
    # track the seq
    i = 1
    
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in data.iterrows():
        tmp_file_name = path+'/'+row['newfilename'] + '_tmp'
        orig_file_name =  path+'/'+row['file'] + row['ext']
        print('rename %s to %s'  % (orig_file_name, tmp_file_name) )
        os.rename(orig_file_name, tmp_file_name)
        tmp_file_names.append(tmp_file_name)
        
    # now remove _tmp files    
    for temp_file in tmp_file_names:
        print('rename %s to %s'  % (temp_file, temp_file.rsplit('_tmp', -1)[0]) )
        os.rename(temp_file, temp_file.rsplit('_tmp', -1)[0]) 
    return

##### splitt the dataset into training and test
here split the dataset into training and testing 
- path : path to the class
- train_path : path to the train dir
- test_path : path to the test dir
- train_files_count : is the number of files in each class to be moved to the training dir
- randomize : either to randomize the files in the class set or not (default is not to randomize)

the output will be copying the data into train and test directories

In [5]:
def split_into_train_test( path, train_path, test_path,train_files_count, randomize=True):
    # read the files in the origianl path ordered by there name
    data = list_files_with_labels( path,randomize)
    # copy the dir name of the class 
    # this only copy the current directory name (without the path)
    dir_name = ntpath.basename(path)
    
    # track the seq
    i = 1
    
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in data.iterrows():
        orig_file_name =  path+'/'+row['file'] + row['ext']
        if (index<train_files_count):
            tmp_file_name = train_path+'/'+row['file']  + row['ext']
        else :
            tmp_file_name = test_path+'/'+row['file']  + row['ext']
       
        print('copy %s to %s'  % (orig_file_name, tmp_file_name) )
        shutil.copy(orig_file_name, tmp_file_name)
    return

##### shuffle  dataset  files

shuffle files in any dataset then rename all the files to _tmp files in order not to overwrite files that already exist

** this can be used to shuffle any directory but the main reason here is to make sure that the  dataset is shuffled in order to avoid overfitting 

In [6]:
def shuffle_dataset(path):
    # first list the files in the input dir and put the porposed name in the dataframe
    data = list_files_with_labels( path,randomize=True)
    # create a new column with the random file selected from the same dataset
    # so the files will be shuffled
    data['newfile'] = data["file"].transform(np.random.permutation)


     
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in data.iterrows():
        tmp_file_name = path+'/'+row['newfile'] + row['ext'] + '_tmp'
        orig_file_name =  path+'/'+row['file'] + row['ext']
        print('rename %s to %s'  % (orig_file_name, tmp_file_name) )
        os.rename(orig_file_name, tmp_file_name)
        tmp_file_names.append(tmp_file_name)
        
    # now remove _tmp f    
    for temp_file in tmp_file_names:
        print('rename %s to %s'  % (temp_file, temp_file.rsplit('_tmp', -1)[0]) )
        os.rename(temp_file, temp_file.rsplit('_tmp', -1)[0])
    return

##### Generate the files / lables csv file
finally generate the csv file that holds the correct mapping of the files and thier correct labels this file should be used as a benchmark against files submitted

In [7]:
def generate_test_csv(test_path, csv_file_name):
    test_dataset = list_files_with_labels( test_path,randomize=True)
    test_dataset['fullfilename'] = test_dataset['file'] + test_dataset['ext']
    header = ["fullfilename", "label"]
    test_dataset.to_csv( csv_file_name, index=False, quoting=3 ,columns = header)
    return

#### create panda from test files dataset
mainly used for the test dataset to generate the file name and the label based on the label criteria randomize is set to true if the files are ranomized in the dataset instead of ordered
will return the panda with the original file name and the new file name 

In [8]:
def list_files_with_xlabels(path, randomize=False):
    dataset = []
    files = sorted(os.listdir(path))
    file = [re.split('[_ \.]+',x) for x in files]
    np.array(file).reshape(len(file),3)
    dataset = pd.DataFrame(np.array(file).reshape(len(file),3), columns =  ['label', 'file', 'ext'] )

    # if randomize is set to True then shuffle the dataset dataframe
    if (randomize) :
        dataset= dataset.sample(frac=1)
        dataset = dataset.sample(frac=1).reset_index(drop=True)

    return dataset

#### shuffle  test dataset  files

shuffle files in the <b>testing dataset </b> then rename all the files to _tmp files in order not to overwrite files that already exist

** this can be used to shuffle any directory but the main reason here is to make sure that the testing dataset is shuffled in order to avoid overfitting

In [9]:
def shuffle_test_dataset(path):
    # first list the files in the input dir and put the porposed name in the dataframe
    data = list_files_with_xlabels( path,randomize=True)

    # recreate index in case the dataset has been randomize
    # so the seq_number will be reflected in out file_name
    df = data.sample(frac=1).reset_index(drop=True)
    
    # new file name will be classid_seq(of seq_len).ext
    # for example 
    # 1_0001.mp3
    # 1_0002.mp3
    
    # find the maximum number of files then use this as a sequence length
    seqlen= len(str(len(df)))
    df['newfilename'] = (df.index + 1).astype(str).str.zfill(seqlen) + '.' +df.ext

    #if not data:
     #   return
    
    # track the seq
    i = 1
    
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in df.iterrows():
        tmp_file_name = path+'/'+row['newfilename'] + '_tmp'
        orig_file_name =  path+'/'+row['label']+'_'+row['file'] +'.'+ row['ext']
        print('rename %s to %s'  % (orig_file_name, tmp_file_name) )
        tmp_file_names.append(tmp_file_name)

        
    # now remove _tmp files    
    for temp_file in tmp_file_names:
        print('rename %s to %s'  % (temp_file, temp_file.rsplit('_tmp', -1)[0]) )
        #os.rename(temp_file, temp_file.rsplit('_tmp', -1)[0]) 
    return df

#### Generate the files / lables csv file
finally generate the csv file that holds the correct mapping of the files and thier correct labels this file should be used as a benchmark against files submitted additionally a new csv file is created with the new and old names in order to keep track of the files.


In [10]:
def generate_test_csv_with_original(path,csv_file_name):
    # gat the panda with orignal and new file names
    data = list_files_with_xlabels( path,randomize=True)
    header = ["file", "label"]
    data.to_csv( csv_file_name, index=False, quoting=3 ,columns = header)
    return

#### Example code
the following sample code could be used to generate train / test dataset from the dataset

-- you can copy it and paste it in the code 

In [14]:
# first of all define the location of the files
print('----Start----')
path='/ds/dataset/quran/'
train_path='/ds/dataset/quran/train'
test_path='/ds/dataset/quran/test'
csv_file_name = '/ds/dataset/quran/test_dataset.csv'

#first rename files in the path to sequenced files prefixed with label 
print('rename file in dataset %s' % (path))

rename_files_dataset(path, 4)

# suffle the files in the path
print('shuffle files in dataset %s' % (path))
shuffle_dataset(path)

# then split into train and test
print('then split into train and test')
split_into_train_test( path, train_path, test_path,20)

# suffle the files in the test
print('shuffle test dataset')
shuffle_test_dataset(test_path)

# write csv file
csv_file_name ='test_dataset.csv'
csv_file= path+'/'+csv_file_name
generate_test_csv(test_path, csv_file)

----Start----
rename file in dataset /ds/dataset/quran/
rename /ds/dataset/quran//reset_ds.sh to /ds/dataset/quran//_0001.sh_tmp
rename /ds/dataset/quran//112 to /ds/dataset/quran//_0002_tmp
rename /ds/dataset/quran//108 to /ds/dataset/quran//_0003_tmp
rename /ds/dataset/quran//103 to /ds/dataset/quran//_0004_tmp
rename /ds/dataset/quran//111 to /ds/dataset/quran//_0005_tmp
rename /ds/dataset/quran//train to /ds/dataset/quran//_0006_tmp
rename /ds/dataset/quran//104 to /ds/dataset/quran//_0007_tmp
rename /ds/dataset/quran//100 to /ds/dataset/quran//_0008_tmp
rename /ds/dataset/quran//107 to /ds/dataset/quran//_0009_tmp
rename /ds/dataset/quran//114 to /ds/dataset/quran//_0010_tmp
rename /ds/dataset/quran//test to /ds/dataset/quran//_0011_tmp
rename /ds/dataset/quran//109 to /ds/dataset/quran//_0012_tmp
rename /ds/dataset/quran//110 to /ds/dataset/quran//_0013_tmp
rename /ds/dataset/quran//Dataset to /ds/dataset/quran//_0014_tmp
rename /ds/dataset/quran//predict to /ds/dataset/quran//_0

IsADirectoryError: [Errno 21] Is a directory: '/ds/dataset/quran//_0008'

In [27]:
input_path= '/ds/dataset/quran/raw'
train_path='/ds/dataset/quran/train'
test_path='/ds/dataset/quran/test'
csv_file_name = '/ds/dataset/quran/test_dataset.csv'

###### list all dataset and convert the file names (

In [28]:
result = []
reg_compile = re.compile("[0-9]")
for dirpath, dirnames, filenames in os.walk(input_path):
    result = result + [dirname for dirname in dirnames if  reg_compile.match(dirname)]
result = sorted(result)

In [29]:
for dir in result:
    # first create the same file structure in train / test
    input_dir = input_path+'/'+dir
    print (input_dir)
    
    train_dir = train_path+'/'+dir
    test_dir = test_path+'/'+dir
    print ('Creating %s' %(train_dir))
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    print ('Creating %s' %(test_dir))
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)


    # then split into train and test
    print('then split into train %s and test %s'% (train_dir,test_dir))
    split_into_train_test( input_dir, train_dir, test_dir,400)
    
    
    print('rename file in train dataset %s' % (train_dir))
    rename_files_dataset(train_dir, 4)
    print('rename file in test dataset %s' % (test_dir))
    rename_files_dataset(test_dir, 4)
    
    
    # suffle the files in the test
    print('shuffle train/test dataset')
    shuffle_test_dataset(test_dir)
    shuffle_test_dataset(train_dir)
    

/ds/dataset/quran/raw/0
Creating /ds/dataset/quran/train/0
Creating /ds/dataset/quran/test/0
then split into train /ds/dataset/quran/train/0 and test /ds/dataset/quran/test/0
copy /ds/dataset/quran/raw/0/100-Al-adiyat_75.mp3 to /ds/dataset/quran/train/0/100-Al-adiyat_75.mp3
copy /ds/dataset/quran/raw/0/100_87.mp3 to /ds/dataset/quran/train/0/100_87.mp3
copy /ds/dataset/quran/raw/0/100_76.mp3 to /ds/dataset/quran/train/0/100_76.mp3
copy /ds/dataset/quran/raw/0/100_uP_bY_mUSLEm.Ettounssi_32.mp3 to /ds/dataset/quran/train/0/100_uP_bY_mUSLEm.Ettounssi_32.mp3
copy /ds/dataset/quran/raw/0/100_uP_bY_mUSLEm.Ettounssi_25.mp3 to /ds/dataset/quran/train/0/100_uP_bY_mUSLEm.Ettounssi_25.mp3
copy /ds/dataset/quran/raw/0/100_uP_bY_mUSLEm.Ettounssi_43.mp3 to /ds/dataset/quran/train/0/100_uP_bY_mUSLEm.Ettounssi_43.mp3
copy /ds/dataset/quran/raw/0/100_165.mp3 to /ds/dataset/quran/train/0/100_165.mp3
copy /ds/dataset/quran/raw/0/100_uP_bY_mUSLEm_17.mp3 to /ds/dataset/quran/train/0/100_uP_bY_mUSLEm_17.mp3

copy /ds/dataset/quran/raw/0/100-Al-adiyat_58.mp3 to /ds/dataset/quran/train/0/100-Al-adiyat_58.mp3
copy /ds/dataset/quran/raw/0/110-An-nasr_100.mp3 to /ds/dataset/quran/train/0/110-An-nasr_100.mp3
copy /ds/dataset/quran/raw/0/100_193.mp3 to /ds/dataset/quran/train/0/100_193.mp3
copy /ds/dataset/quran/raw/0/100_135.mp3 to /ds/dataset/quran/train/0/100_135.mp3
copy /ds/dataset/quran/raw/0/100_adiyat_uP_bY_mUSLEm.mp3 to /ds/dataset/quran/train/0/100_adiyat_uP_bY_mUSLEm.mp3
copy /ds/dataset/quran/raw/0/100_M.Ettounssi.mp3 to /ds/dataset/quran/train/0/100_M.Ettounssi.mp3
copy /ds/dataset/quran/raw/0/100-Al-adiyat_15.mp3 to /ds/dataset/quran/train/0/100-Al-adiyat_15.mp3
copy /ds/dataset/quran/raw/0/100_58.mp3 to /ds/dataset/quran/train/0/100_58.mp3
copy /ds/dataset/quran/raw/0/100_uP_bY_mUSLEm_6.mp3 to /ds/dataset/quran/train/0/100_uP_bY_mUSLEm_6.mp3
copy /ds/dataset/quran/raw/0/100-Al-adiyat_47.mp3 to /ds/dataset/quran/train/0/100-Al-adiyat_47.mp3
copy /ds/dataset/quran/raw/0/100_231.mp3 t

copy /ds/dataset/quran/raw/0/100_162.mp3 to /ds/dataset/quran/train/0/100_162.mp3
copy /ds/dataset/quran/raw/0/100_47.mp3 to /ds/dataset/quran/train/0/100_47.mp3
copy /ds/dataset/quran/raw/0/100_86.mp3 to /ds/dataset/quran/train/0/100_86.mp3
copy /ds/dataset/quran/raw/0/muslem.ettounsi-100.mp3 to /ds/dataset/quran/train/0/muslem.ettounsi-100.mp3
copy /ds/dataset/quran/raw/0/100_214.mp3 to /ds/dataset/quran/train/0/100_214.mp3
copy /ds/dataset/quran/raw/0/100_uP_bY_mUSLEm.Ettounssi_11.mp3 to /ds/dataset/quran/train/0/100_uP_bY_mUSLEm.Ettounssi_11.mp3
copy /ds/dataset/quran/raw/0/100_92.mp3 to /ds/dataset/quran/train/0/100_92.mp3
copy /ds/dataset/quran/raw/0/100_83.mp3 to /ds/dataset/quran/train/0/100_83.mp3
copy /ds/dataset/quran/raw/0/100_uP_bY_mUSLEm_29.mp3 to /ds/dataset/quran/train/0/100_uP_bY_mUSLEm_29.mp3
copy /ds/dataset/quran/raw/0/100_122.mp3 to /ds/dataset/quran/train/0/100_122.mp3
copy /ds/dataset/quran/raw/0/Khalaf-3n-7amzah_100.mp3 to /ds/dataset/quran/train/0/Khalaf-3n-7am

copy /ds/dataset/quran/raw/0/100_114.mp3 to /ds/dataset/quran/test/0/100_114.mp3
copy /ds/dataset/quran/raw/0/108_100.mp3 to /ds/dataset/quran/test/0/108_100.mp3
copy /ds/dataset/quran/raw/0/Khallad_100.mp3 to /ds/dataset/quran/test/0/Khallad_100.mp3
copy /ds/dataset/quran/raw/0/100-Al-adiyat_28.mp3 to /ds/dataset/quran/test/0/100-Al-adiyat_28.mp3
copy /ds/dataset/quran/raw/0/100_uP_bY_mUSLEm_33.mp3 to /ds/dataset/quran/test/0/100_uP_bY_mUSLEm_33.mp3
copy /ds/dataset/quran/raw/0/100_227.mp3 to /ds/dataset/quran/test/0/100_227.mp3
copy /ds/dataset/quran/raw/0/100_256.mp3 to /ds/dataset/quran/test/0/100_256.mp3
copy /ds/dataset/quran/raw/0/100-Al-adiyat_53.mp3 to /ds/dataset/quran/test/0/100-Al-adiyat_53.mp3
rename file in train dataset /ds/dataset/quran/train/0
rename /ds/dataset/quran/train/0/100_145.mp3 to /ds/dataset/quran/train/0/0_0001.mp3_tmp
rename /ds/dataset/quran/train/0/100_uP_bY_mUSLEm.Ettounssi_8.mp3 to /ds/dataset/quran/train/0/0_0002.mp3_tmp
rename /ds/dataset/quran/train

rename /ds/dataset/quran/test/0/0_0059.mp3_tmp to /ds/dataset/quran/test/0/0_0059.mp3
rename /ds/dataset/quran/test/0/0_0060.mp3_tmp to /ds/dataset/quran/test/0/0_0060.mp3
rename /ds/dataset/quran/test/0/0_0061.mp3_tmp to /ds/dataset/quran/test/0/0_0061.mp3
rename /ds/dataset/quran/test/0/0_0062.mp3_tmp to /ds/dataset/quran/test/0/0_0062.mp3
rename /ds/dataset/quran/test/0/0_0063.mp3_tmp to /ds/dataset/quran/test/0/0_0063.mp3
rename /ds/dataset/quran/test/0/0_0064.mp3_tmp to /ds/dataset/quran/test/0/0_0064.mp3
rename /ds/dataset/quran/test/0/0_0065.mp3_tmp to /ds/dataset/quran/test/0/0_0065.mp3
rename /ds/dataset/quran/test/0/0_0066.mp3_tmp to /ds/dataset/quran/test/0/0_0066.mp3
rename /ds/dataset/quran/test/0/0_0067.mp3_tmp to /ds/dataset/quran/test/0/0_0067.mp3
rename /ds/dataset/quran/test/0/0_0068.mp3_tmp to /ds/dataset/quran/test/0/0_0068.mp3
rename /ds/dataset/quran/test/0/0_0069.mp3_tmp to /ds/dataset/quran/test/0/0_0069.mp3
rename /ds/dataset/quran/test/0/0_0070.mp3_tmp to /ds/

...

copy /ds/dataset/quran/raw/9/109_109.mp3 to /ds/dataset/quran/train/9/109_109.mp3
copy /ds/dataset/quran/raw/9/109_205.mp3 to /ds/dataset/quran/train/9/109_205.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm_15.mp3 to /ds/dataset/quran/train/9/109_uP_bY_mUSLEm_15.mp3
copy /ds/dataset/quran/raw/9/109_238.mp3 to /ds/dataset/quran/train/9/109_238.mp3
copy /ds/dataset/quran/raw/9/109_211.mp3 to /ds/dataset/quran/train/9/109_211.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm.Ettounssi_13.mp3 to /ds/dataset/quran/train/9/109_uP_bY_mUSLEm.Ettounssi_13.mp3
copy /ds/dataset/quran/raw/9/109_169.mp3 to /ds/dataset/quran/train/9/109_169.mp3
copy /ds/dataset/quran/raw/9/109_141.mp3 to /ds/dataset/quran/train/9/109_141.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm.Ettounssi_29.mp3 to /ds/dataset/quran/train/9/109_uP_bY_mUSLEm.Ettounssi_29.mp3
copy /ds/dataset/quran/raw/9/109-Alkafirun-.mp3 to /ds/dataset/quran/train/9/109-Alkafirun-.mp3
copy /ds/dataset/quran/raw/9/109_muslem.ettounsi_3.mp3

copy /ds/dataset/quran/raw/9/109_224.mp3 to /ds/dataset/quran/train/9/109_224.mp3
copy /ds/dataset/quran/raw/9/109-Alkafirun.mp3 to /ds/dataset/quran/train/9/109-Alkafirun.mp3
copy /ds/dataset/quran/raw/9/109_58.mp3 to /ds/dataset/quran/train/9/109_58.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm.Ettounssi_40.mp3 to /ds/dataset/quran/train/9/109_uP_bY_mUSLEm.Ettounssi_40.mp3
copy /ds/dataset/quran/raw/9/109_125.mp3 to /ds/dataset/quran/train/9/109_125.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm.mp3 to /ds/dataset/quran/train/9/109_uP_bY_mUSLEm.mp3
copy /ds/dataset/quran/raw/9/109-Al-kafiroon_58.mp3 to /ds/dataset/quran/train/9/109-Al-kafiroon_58.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm.Ettounssi.mp3 to /ds/dataset/quran/train/9/109_uP_bY_mUSLEm.Ettounssi.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm.Ettounssi_34.mp3 to /ds/dataset/quran/train/9/109_uP_bY_mUSLEm.Ettounssi_34.mp3
copy /ds/dataset/quran/raw/9/109_176.mp3 to /ds/dataset/quran/train/9/109_176.mp3
copy

copy /ds/dataset/quran/raw/9/Hisham-Ibn-Aamir_109.mp3 to /ds/dataset/quran/train/9/Hisham-Ibn-Aamir_109.mp3
copy /ds/dataset/quran/raw/9/Rowais_109.mp3 to /ds/dataset/quran/train/9/Rowais_109.mp3
copy /ds/dataset/quran/raw/9/109_210.mp3 to /ds/dataset/quran/train/9/109_210.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm.Ettounssi_20.mp3 to /ds/dataset/quran/train/9/109_uP_bY_mUSLEm.Ettounssi_20.mp3
copy /ds/dataset/quran/raw/9/muslem.ettounsi-109_1.mp3 to /ds/dataset/quran/train/9/muslem.ettounsi-109_1.mp3
copy /ds/dataset/quran/raw/9/109_120.mp3 to /ds/dataset/quran/train/9/109_120.mp3
copy /ds/dataset/quran/raw/9/109_255.mp3 to /ds/dataset/quran/train/9/109_255.mp3
copy /ds/dataset/quran/raw/9/109_46.mp3 to /ds/dataset/quran/train/9/109_46.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm_52.mp3 to /ds/dataset/quran/train/9/109_uP_bY_mUSLEm_52.mp3
copy /ds/dataset/quran/raw/9/109-_vbr.mp3 to /ds/dataset/quran/train/9/109-_vbr.mp3
copy /ds/dataset/quran/raw/9/109_uP_bY_mUSLEm_47.mp3

rename /ds/dataset/quran/train/9/109 - Al-Kafiroon.mp3 to /ds/dataset/quran/train/9/9_0185.mp3_tmp
rename /ds/dataset/quran/train/9/109-Al-kafiroon_14.mp3 to /ds/dataset/quran/train/9/9_0186.mp3_tmp
rename /ds/dataset/quran/train/9/109-Up-By-MusleM.mp3 to /ds/dataset/quran/train/9/9_0187.mp3_tmp
rename /ds/dataset/quran/train/9/Qaloon_109.mp3 to /ds/dataset/quran/train/9/9_0188.mp3_tmp
rename /ds/dataset/quran/train/9/109-uP_bY_mUSLEm.eTTounssi.mp3 to /ds/dataset/quran/train/9/9_0189.mp3_tmp
rename /ds/dataset/quran/train/9/109_190.mp3 to /ds/dataset/quran/train/9/9_0190.mp3_tmp
rename /ds/dataset/quran/train/9/109_239.mp3 to /ds/dataset/quran/train/9/9_0191.mp3_tmp
rename /ds/dataset/quran/train/9/109_120.mp3 to /ds/dataset/quran/train/9/9_0192.mp3_tmp
rename /ds/dataset/quran/train/9/109_uP_bY_mUSLEm_36.mp3 to /ds/dataset/quran/train/9/9_0193.mp3_tmp
rename /ds/dataset/quran/train/9/109_180.mp3 to /ds/dataset/quran/train/9/9_0194.mp3_tmp
rename /ds/dataset/quran/train/9/109_168.mp3 t

rename /ds/dataset/quran/train/9/022.mp3_tmp to /ds/dataset/quran/train/9/022.mp3
rename /ds/dataset/quran/train/9/023.mp3_tmp to /ds/dataset/quran/train/9/023.mp3
rename /ds/dataset/quran/train/9/024.mp3_tmp to /ds/dataset/quran/train/9/024.mp3
rename /ds/dataset/quran/train/9/025.mp3_tmp to /ds/dataset/quran/train/9/025.mp3
rename /ds/dataset/quran/train/9/026.mp3_tmp to /ds/dataset/quran/train/9/026.mp3
rename /ds/dataset/quran/train/9/027.mp3_tmp to /ds/dataset/quran/train/9/027.mp3
rename /ds/dataset/quran/train/9/028.mp3_tmp to /ds/dataset/quran/train/9/028.mp3
rename /ds/dataset/quran/train/9/029.mp3_tmp to /ds/dataset/quran/train/9/029.mp3
rename /ds/dataset/quran/train/9/030.mp3_tmp to /ds/dataset/quran/train/9/030.mp3
rename /ds/dataset/quran/train/9/031.mp3_tmp to /ds/dataset/quran/train/9/031.mp3
rename /ds/dataset/quran/train/9/032.mp3_tmp to /ds/dataset/quran/train/9/032.mp3
rename /ds/dataset/quran/train/9/033.mp3_tmp to /ds/dataset/quran/train/9/033.mp3
rename /ds/datas

In [31]:
# write csv file
generate_test_csv_with_original(test_path, csv_file_name)

ValueError: cannot reshape array of size 15 into shape (15,3)